## RAG(Retrieval-Augmented Generation): 검색 증강 생성
* RAG는 미리 지정한 텍스트를 데이터베이스로 준비해 두었다가 사용자가 입력하면 그 입력 내용과 연관성이 높은 텍스트를 데이터베이스에서 검색해 프롬프트에 추가해 보다 정확한 답변을 할 수 있게 하는 기법
* 질문에 더 정확하고 풍부한 답변을 주기 위해 정보 검색과 답변 생성을 결합한 기술
* 정보 검색 단계: 사용자가 질문을 하면, 외부 데이터베이스나 문서에서 관련 정보 검색
* 답변 생성 단계: 찾은 정보를 바탕으로 AI 모델이 답변 생성

# 임베딩과 벡터 DB
* 지정한 텍스트를 데이터베이스에 저장하기 위해서는 먼저 문자를 숫자로 변환하는 임베딩(embedding)이 필요하며
* 데이터를 저장하는 데이터베이스는 RDBMS가 아닌 벡터 검색에 특화된 vectorDB를 이용해야 한다.

최근 인공지능과 NLP 애플리케이션에서 **Vector Database (vectordb)**는 빠르게 인기를 얻고 있다. 이는 텍스트, 이미지, 오디오 등의 데이터를 벡터 형식으로 저장하고 검색할 수 있는 데이터베이스로, 특히 **임베딩 벡터**를 사용해 의미 기반 검색을 수행한다.

### 대표적인 Vector Database
1. **FAISS (Facebook AI Similarity Search)**:
   - **Facebook AI**에서 개발한 오픈 소스 라이브러리.
   - 매우 빠른 유사도 검색과 군집화 기능을 제공.
   - GPU 가속을 지원해 대량의 데이터를 효율적으로 처리할 수 있음.

2. **Milvus**:
   - **Zilliz**에서 개발한 오픈 소스 vectordb로, 높은 성능과 확장성을 자랑함.
   - 벡터 검색과 혼합 검색(hybrid search)을 지원해 다양한 유형의 데이터를 처리할 수 있음.
   - 분산 시스템을 통해 대규모 데이터셋에서도 효율적인 검색이 가능.

3. **Weaviate**:
   - 의미론적 검색을 위해 설계된 오픈 소스 vectordb.
   - 데이터베이스 내에서 벡터를 자동으로 생성하거나 기존의 임베딩을 사용할 수 있음.
   - 다양한 NLP 모델과의 통합을 지원.

4. **Pinecone**:
   - 클라우드 기반 벡터 데이터베이스 서비스로, 사용자가 인프라를 직접 관리할 필요 없이 벡터 검색 기능을 제공.
   - 쉽게 확장 가능하고 API 기반으로 간편하게 벡터 데이터를 관리할 수 있음.
   - 지리적으로 분산된 클러스터를 제공해 글로벌 검색 성능을 최적화함.

### Vector Database의 특징
- **의미 기반 검색**: 단순한 키워드 매칭이 아닌, 벡터 간의 거리(예: 코사인 유사도)를 이용해 데이터의 의미적 유사성을 파악.
- **빠른 검색 속도**: 수백만에서 수십억 개의 벡터에 대한 검색을 실시간으로 처리할 수 있음.
- **확장성**: 많은 vectordb는 분산 처리와 클러스터링을 통해 대규모 데이터를 효율적으로 관리할 수 있음.

### 주요 사용 사례
- **추천 시스템**: 사용자 선호도를 분석하고 유사한 제품이나 콘텐츠를 추천.
- **챗봇 및 QA 시스템**: 의미적으로 유사한 질문과 답변을 매칭하여 더 자연스러운 대화와 검색을 가능하게 함.
- **이미지 검색**: 이미지의 특징 벡터를 사용해 시각적 유사도를 기반으로 검색.
- **문서 검색**: 임베딩 벡터를 사용하여 의미적으로 관련된 문서나 내용을 빠르게 찾음.

Vector Database는 AI와 NLP 애플리케이션의 핵심 기술로 자리잡고 있으며, 데이터의 의미를 기반으로 한 고속 검색과 처리가 필요할 때 필수적이다.


In [1]:
import os
import pandas as pd
from google import genai

In [2]:
with open('./data/hotel_data.txt', 'r', encoding='utf-8') as f:
    data = f.read()
print(data)

1. 손님 맞이
손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다.

2. 체크인과 체크아웃
체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.

3. Wi-Fi 및 주차장 안내
모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.

4. 배리어 프리 대응
유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.

5. 반려동물 대응
반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.

6. 룸 서비스
오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의 협력도 중요하다.

7. 금연 정책 및 흡연실 안내
모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실을 마련한다. 이 정보를 명확하게 전달하고, 흡연실 위치와 이용 시간을 고객에게 안내해 주어야 한다.

8. 취소 정책
취소 수

In [3]:
## 텍스트 -> 각각의 항목에 맞춰서 내용을 따로 나누기(기존의 하나의 문장을 나누기)
data = data.split('\n\n')

In [4]:
# 카테고리와 카테고리에 맞는 메뉴얼로 만듦
data[0].split('\n')

['1. 손님 맞이',
 "손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다."]

In [5]:
text2df = {}
for content in data:
    temp = content.split('\n')
    text2df.setdefault('title', []).append(temp[0])
    text2df.setdefault('content', []).append(temp[1])
    
#     print(content)
#     print(content.split('\n')[0]) # 메뉴얼 제목만 나옴
#     print(content.split('\n')[1]) # 메뉴얼 내용만 나옴

In [6]:
df = pd.DataFrame(text2df)
df

,title,content
0,1. 손님 맞이,손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이...
1,2. 체크인과 체크아웃,"체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게..."
2,3. Wi-Fi 및 주차장 안내,모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수...
3,4. 배리어 프리 대응,"유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록..."
4,5. 반려동물 대응,"반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을..."
5,6. 룸 서비스,오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에...
6,7. 금연 정책 및 흡연실 안내,모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실...
7,8. 취소 정책,"취소 수수료는 전날까지 연락 시 숙박 요금의 30%, 당일 취소 시 50%, 연락 ..."
8,9. 결제 방법,"체크아웃 시 프런트에서 현금, 신용카드, 직불카드로 결제한다. 또한 인터넷 예약을 ..."
9,10. 항상 존중을 실천한다.,"고객 한 사람 한 사람을 존중하는 태도로 대하자. 고객에 대한 예의, 배려, 전문성..."


# 텍스트 임베딩
- tiktoken : OpenAI의 GPT 모델에서 사용하는 토큰화 라이브러리
    - 토큰 : LLM에서 사용하는 단위(약 1.5단어 정도)
    - 입력된 텍스트를 모델이 이해할 수 있는 토큰 단위로 불리하거나 토큰 수를 계산 시 사용
    - 텍스트의 길이를 정확히 파악하고 모델의 입출력 제한을 관리

In [32]:
# !pip install tiktoken

In [33]:
import tiktoken
from openai import OpenAI

In [36]:
# client = OpenAI()

# # 임배딩 매개변수 설정
# embadding_model = 'text-embedding-3-small'
# embedding_encoding = 'cl100k_base'
# max_tokens = 100

# tokenizer = tiktoken.get_encoding(embedding_encoding)
# df['n_tokens'] = df['content'].apply(lambda x: len(tokenizer.encode(x)))
# df

In [47]:
# content를 임베딩
def get_embedding(text,model):
    text = text.replace('\n', '')
    return client.embeddings.create(input=[text], model=model).data[0].embedding
    

In [ ]:
df['embeddings'] = df['content'].apply(lambda x: get_embedding(x, model=embedding_model))
df

## RAG 이용한 GPT 생성


In [48]:
def create_context(question, df, max_len=1800):
    # 질문 벡터화
    q_embeddings = client.embeddings.create(input=[question], model=embedding_model).data[0]['embedding']
    # 질문 RAG 비교 후 코사인 유사도 산출 후 distance 컬럼에 저장
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].apply(eval).apply(np.array).value)
    
    # 컨텍스트 저장 위한 리스트
    returns = []
    
    # 컨텍스트 현재 길이
    cur_len = 0
    
    # 학습 데이터 유사도 순으로 정렬하고 토큰 개수 한도까지 컨텍스트에 추가
    for _, row in df.sort_values('distances', ascending=True).iterrows():
        # 텍스트 길이를 현재 길이 더하기
        cur_len += row['n_tokens'] + 4
        
        # 텍스트 너무 길면 종료
        if cur_len > max_len:
            break
        
        # 컨텍스트 목록에 텍스트 추가
        returns.append(row['text'])
    
return "\n\n###\n\n".join(returns)


def answer_question(question, conversation_history):
    
    # RAG 데이터 불러오기
    df = pd.read_csv('embedding.csv')
    
    # 질문과 RAG 데이터 비교해 컨텍스트 생성
    context = create_context(question, df, max_len=200)
    
    # 프롬프트 생성하고 대화 기록에 추가
    prompt = f"당신은 어느 호텔의 직원입니다. 문맥에 따라 고객의 질문에 정중하게 답해주세요.\n
                컨텍스트가 질문에 대답할 수 없는 경우 '모르겠습니다'라고 대답하세요.\n
                \n컨텍스트: {context}\n\n---\n\n질문: {question}\n답변:"
    conversation_history.append({'role': "user", 'content': prompt})
    
    try:
        # GPT에서 답변 생성
        response = client.chat.completions.create(
                    model = 'gpt-4o-mini',
                    message=conversation_history,
                    temperature=1)
        return reponse.choices[0].message.content.strip()
    
    except Exception as e:
        print(e)
        return '오류'


def distance_from_embeddings:
    distance_metrics = {
        'cosine' : spatial.distance.cosine,
        'L1' : spatial.distance.cityblock,
        'L2' : spatial.distance.eucliden,
        'Linf' : spatial.distance.chebyshev
    }
    
    distance = [distance_metrics[distance_metrics](query_embedding, embedding) for embedding in embedding]
    return distance


## 실행 부분


In [ ]:
# !pip install faiss-cpu

## Gemini 활용 구현
---

### 텍스트 임베딩
- tiktoken : OpenAI의 GPT 모델에서 사용하는 토큰화 라이브러리
    - 토큰 : LLM에서 사용하는 단위(약 1.5단어 정도)
    - 입력된 텍스트를 모델이 이해할 수 있는 토큰 단위로 불리하거나 토큰 수를 계산 시 사용
    - 텍스트의 길이를 정확히 파악하고 모델의 입출력 제한을 관리


In [7]:
# !pip install pandas google-generativeai

In [8]:
import google.generativeai as genai

api_key = os.getenv('GEMINAI_API_KEY')
genai.configure(api_key=api_key)

# 'gemini-1.5-flash' 모델을 사용하여 텍스트 토큰 수를 계산합니다.
# count_tokens API 호출에 요금이 청구되지 않며 할당량 제한도 없음
generation_model = genai.GenerativeModel('gemini-1.5-flash')
token_counting_model = generation_model

df['n_tokens'] = df['content'].apply(lambda text:token_counting_model.count_tokens(contents=[{"text": text}]).total_tokens)
df


C:\Users\Admin\miniforge3\envs\ai_seving\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,title,content,n_tokens
0,1. 손님 맞이,손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이...,92
1,2. 체크인과 체크아웃,"체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게...",95
2,3. Wi-Fi 및 주차장 안내,모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수...,81
3,4. 배리어 프리 대응,"유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록...",72
4,5. 반려동물 대응,"반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을...",94
5,6. 룸 서비스,오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에...,84
6,7. 금연 정책 및 흡연실 안내,모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실...,71
7,8. 취소 정책,"취소 수수료는 전날까지 연락 시 숙박 요금의 30%, 당일 취소 시 50%, 연락 ...",80
8,9. 결제 방법,"체크아웃 시 프런트에서 현금, 신용카드, 직불카드로 결제한다. 또한 인터넷 예약을 ...",73
9,10. 항상 존중을 실천한다.,"고객 한 사람 한 사람을 존중하는 태도로 대하자. 고객에 대한 예의, 배려, 전문성...",73


In [9]:
# content를 임베딩
def get_embedding(text,model):
    text = text.replace('\n', ' ')
    response = genai.embed_content(model=model, content=text, task_type="RETRIEVAL_DOCUMENT")
    return response['embedding']

In [10]:
# 임배딩 매개변수 설정
embedding_model = 'embedding-001'
df['embeddings'] = df['content'].apply(lambda x: get_embedding(x, model=embedding_model))
df

,title,content,n_tokens,embeddings
0,1. 손님 맞이,손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이...,92,"[0.047990974, 0.0022215373, -0.049586423, -0.0..."
1,2. 체크인과 체크아웃,"체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게...",95,"[0.045965537, -0.0012585014, -0.05666608, -0.0..."
2,3. Wi-Fi 및 주차장 안내,모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수...,81,"[0.049213726, -0.01923195, -0.05040864, -0.014..."
3,4. 배리어 프리 대응,"유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록...",72,"[0.05669855, 0.0067103114, -0.04471087, -0.001..."
4,5. 반려동물 대응,"반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을...",94,"[0.0566528, 0.0057623824, -0.04597638, -0.0022..."
5,6. 룸 서비스,오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에...,84,"[0.038001366, -0.0011839091, -0.055285856, -0...."
6,7. 금연 정책 및 흡연실 안내,모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실...,71,"[0.04873405, -0.0057860888, -0.053273458, -0.0..."
7,8. 취소 정책,"취소 수수료는 전날까지 연락 시 숙박 요금의 30%, 당일 취소 시 50%, 연락 ...",80,"[0.047183372, -0.0153624965, -0.044350553, -0...."
8,9. 결제 방법,"체크아웃 시 프런트에서 현금, 신용카드, 직불카드로 결제한다. 또한 인터넷 예약을 ...",73,"[0.05647212, 0.0022469126, -0.046772905, -0.00..."
9,10. 항상 존중을 실천한다.,"고객 한 사람 한 사람을 존중하는 태도로 대하자. 고객에 대한 예의, 배려, 전문성...",73,"[0.05795362, 0.003422032, -0.047266956, -0.005..."


In [11]:
df.to_csv('embedding.csv', index=False, encoding='utf-8')

In [18]:
# !pip install scipy

In [17]:
import numpy as np
from numpy.linalg import norm
from scipy import spatial
import ast
import gradio as gr

In [13]:
def create_context(question, df, max_len=1800):
    
    # 질문 벡터화
    q_embeddings_response = genai.embed_content(model=embedding_model, content=question, task_type="RETRIEVAL_QUERY")
    q_embeddings = np.array(q_embeddings_response['embedding'])
    
    # 질문 RAG 비교 후 코사인 유사도 산출 후 distances 컬럼에 저장
    # 두 벡터 간 코사인 유사도 계산 함수
    def cosine_similarity(vec1, vec2):
        if not isinstance(vec1, np.ndarray):
            vec1 = np.array(vec1)
        if not isinstance(vec2, np.ndarray):
            vec2 = np.array(vec2)

        if norm(vec1) == 0 or norm(vec2) == 0:
            return 0.0

        return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))
    
    # 코사인 유사도 산출 후 distances 컬럼에 저장
    df['distances'] = df['embeddings'].apply(lambda doc_emb: cosine_similarity(q_embeddings, np.array(doc_emb)))
    
    # 최종 컨텍스트에 포함될 문서들을 저장할 리스트
    returns = []
    
    # 현재 컨텍스트의 총 토큰 길이
    cur_len = 0
    
    # 유사도 내림차순 정렬
    sorted_df = df.sort_values('distances', ascending=False)
    
    for _, row in sorted_df.iterrows():
        
        # 최대 길이 초과하면 중단
        if cur_len + row['n_tokens'] + 4 > max_len:
            break
        
        # 초과하지 않으면 현재 길이를 업데이트하고 문서 내용을 컨텍스트 리스트에 추가
        cur_len += row['n_tokens'] + 4
        returns.append(row['content'])
    
    return "\n\n###\n\n".join(returns)


In [37]:
def answer_question(question, conversation_history):
    
    # RAG 데이터 불러오기
    df = pd.read_csv('embedding.csv')
    
    # 문자열을 파이썬 리터럴로 평가
    df['embeddings'] = df['embeddings'].apply(ast.literal_eval)
    
    # NumPy 배열로 다시 변환
    df['embeddings'] = df['embeddings'].apply(np.array)
    
    # 질문과 RAG 데이터 비교해 컨텍스트 생성
    context = create_context(question, df, max_len=200)
    
    # Gemini API의 contents 형식에 맞게 대화 기록을 변환
    gemini_formatted_history = []
    
    for entry in conversation_history:
        role = "model" if entry['role'] == "assistant" else entry['role']
        gemini_formatted_history.append({'role': role, 'parts': [{'text': entry['content']}]})
        
    # 프롬프트 생성하고 대화 기록에 추가
    prompt = (
        f"당신은 어느 호텔의 직원입니다. 문맥에 따라 고객의 질문에 정중하게 답해주세요.\n"
        f"컨텍스트가 질문에 대답할 수 없는 경우 '모르겠습니다'라고 대답하세요.\n\n"
        f"컨텍스트: {context}\n\n---\n\n질문: {question}\n답변:"
    )
    gemini_formatted_history.append({'role': "user", 'parts': [{'text': prompt}]})
    
    try:
        # gemini에서 답변 생성
        response = generation_model.generate_content(
            contents=gemini_formatted_history,
            generation_config=genai.types.GenerationConfig(temperature=0.6)
        )
        return response.text.strip()
    except Exception as e:
        print(f"Gemini 답변 생성 중 오류 발생: {e}")
        return '죄송합니다. 현재 질문에 답변할 수 없습니다.'
    

In [38]:
def distance_from_embeddings(query_embedding: np.ndarray, 
                             document_embeddings: list[np.ndarray], 
                             metric: str = 'cosine') -> list[float]:
    distance_metrics = {
        'cosine': spatial.distance.cosine,
        'L1': spatial.distance.cityblock,
        'L2': spatial.distance.euclidean,
        'Linf': spatial.distance.chebyshev}
    
    if metric not in distance_metrics:
        raise ValueError(f"지원하지 않는 거리 지표 '{metric}'입니다. 'cosine', 'L1', 'L2', 'Linf' 중 하나를 선택하세요.")
    selected_distance_func = distance_metrics[metric]
    query_embedding = np.array(query_embedding)
    distances = [selected_distance_func(query_embedding, np.array(doc_emb)) for doc_emb in document_embeddings]

    return distances
    

## 실행 부분

In [39]:
def chat_interface(message, history):
    conversation_history_formatted  = []
    for human_msg, ai_msg in history:
        conversation_history_formatted.append({"role": "user", "content": human_msg})
        conversation_history_formatted.append({"role": "assistant", "content": ai_msg})
    bot_message = answer_question(message, conversation_history_formatted)
    return bot_message

In [40]:
demo = gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="궁금한 점을 입력하세요...", container=False, scale=7),
    title="🏨 호텔 챗봇",
    description="호텔에 대해 궁금한 점이 있으시면 무엇이든 물어보세요! (예: 입실 시간, 조식 시간, 수영장 위치)", # UI 설명
    examples=[ 
        "입실 시간은 몇 시부터인가요?",
        "조식은 어디서 먹을 수 있나요?",
        "수영장은 몇 시까지 이용 가능한가요?"
    ],
    theme="soft")

# Gradio 데모 실행
demo.launch(share=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17944\681431327.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
C:\Users\Admin\miniforge3\envs\ai_seving\Lib\site-packages\gradio\chat_interface.py:322: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://c96072ebf92d55b89a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
